# Wk 2. Walmart, Rik's sandbox

TODO:
* V clean up date/index shit
* ... submit results
* ... clean up code
* V graph plot submission numbers into known data for evaluation


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from keras.layers import Dense, Activation, BatchNormalization, Dropout
import keras.models
from keras import regularizers
from keras import optimizers

import re, datetime, time
from sklearn.model_selection import train_test_split

In [ ]:
print(datetime.datetime.now())

In [ ]:
def add_datepart(df, fldname, drop=True):
    """add_datepart converts a column of df from a datetime64 to many columns containing
    the information from the date. This applies changes inplace.

    Parameters:
    -----------
    df: A pandas data frame. df gain several new columns.
    fldname: A string that is the name of the date column you wish to expand.
        If it is not a datetime64 series, it will be converted to one with pd.to_datetime.
    drop: If true then the original date column will be removed.

    Examples:
    ---------

    >>> df = pd.DataFrame({ 'A' : pd.to_datetime(['3/11/2000', '3/12/2000', '3/13/2000'], infer_datetime_format=False) })
    >>> df

        A
    0   2000-03-11
    1   2000-03-12
    2   2000-03-13

    >>> add_datepart(df, 'A')
    >>> df

        AYear AMonth AWeek ADay ADayofweek ADayofyear AIs_month_end AIs_month_start AIs_quarter_end AIs_quarter_start AIs_year_end AIs_year_start AElapsed
    0   2000  3      10    11   5          71         False         False           False           False             False        False          952732800
    1   2000  3      10    12   6          72         False         False           False           False             False        False          952819200
    2   2000  3      11    13   0          73         False         False           False           False             False        False          952905600
    
    Acknowledgment: Fast.ai ML library https://github.com/fastai/fastai/blob/master/fastai/structured.py
    Jeremy howard c.s.
    """
# Rik disabled a few attributes because our dates are Fridays only
#               'Dayofweek', 
#               'Day', 
#               'Is_month_end', 
#               'Is_month_start', 
#               'Is_quarter_end', 
#               'Is_quarter_start', 
#               'Is_year_end', 
#               'Is_year_start'

    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ('Year', 
              'Month', 
              'Week', 
              'Dayofyear',
             ):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldname, axis=1, inplace=True)


## load data from CSV and organize

In [ ]:
# @rik: the only reason you indexed Date is to produce a plot
# with blanks in it at all the right places

# dateparse = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date()
# , date_parser=dateparse
# , parse_dates=['Date']
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
# train.sort_values(['Date'], inplace=True)
# test.sort_values(['Date'], inplace=True)

len_train = len(train)
len_test = len(test)

df = pd.concat([train,test],axis=0) # Join train and test
pristine = df.copy()
# df['Date'] = pd.to_datetime(df['Date'])  ## FIXME if .fit() croaks
pristine['Date'] = pd.to_datetime(df['Date'])
#### DO NOT SORT concatenated df: this will make the cat irreversible!!!!!!!!!!!!!!!!!!!!!!!


In [ ]:
df.head(2)

In [ ]:
# engineer this:
# list of KNOWN holidays
# for all dates in our range, search nearest holiday: remember its name/date
# for all dates, compute distance (pos or neg) to nearest holiday
# engineer the distance into a number that goes high as HD nears and goes low as HD passes
# clip this number to about 3 weeks
# fill new column (named 'Nearest_HD') in df with name of nearest holiday
# dummify the Nearest_HD column
# fill column Nearest_HD in df with distance number


In [ ]:
def nearest_hd(d):
    us_holidays = [
        'xmas09',
        'sbowl10',
        'lday10',
        'tnxg10',
        'xmas10',
        'sbowl11',
        'lday11',
        'tnxg11',
        'xmas11',
        'sbowl12',
        'lday12',
        'tnxg12',
    ]

    us_holidates = [
        pd.to_datetime("2009, 12, 25"),
        pd.to_datetime("2010,  2,  7"),
        pd.to_datetime("2010,  9,  6"),
        pd.to_datetime("2010, 11, 25"),
        pd.to_datetime("2010, 12, 25"),
        pd.to_datetime("2011,  2,  6"),
        pd.to_datetime("2011,  9,  5"),
        pd.to_datetime("2011, 11, 24"),
        pd.to_datetime("2011, 12, 25"),
        pd.to_datetime("2012,  2,  5"),
        pd.to_datetime("2012,  9,  3"),
        pd.to_datetime("2012, 11, 22"),
    ]
    
    shortest = (datetime.datetime(2010,12,31) - datetime.datetime(2001,1,1)) # kick off with a 10 year distance
    nearest = len(us_holidates)
    
    for h in range(len(us_holidates)):
        dist = (us_holidates[h] - d)
        if abs(dist) <= abs(shortest):
            shortest = dist
            nearest = us_holidays[h]
    return(nearest, shortest.days)

# de volgende twee cellen nog samenvoegen @rik

In [ ]:
# for all dates in our range, search nearest holiday: remember its name/date
# for all dates, compute distance (pos or neg) to nearest holiday
# engineer the distance into a number that goes far negative as HD nears and
# goes up positive at HD and lowers back to zero as HD passes
# clip this number to about 3 weeks

# all days in range dataframe (column 'Date')
alldays = pd.unique(df['Date'])
holi_dist = {}  # new dict
all_holidays = [] # new array

# iterate over all days;the disorfderof alldaysis irrelevant (right?!)
for d in alldays:
    # compute distance proper
    nearest, distance = nearest_hd( pd.to_datetime(d) )   # , infer_datetime_format=True
    # engineer some number that starts at 0, goes negative as HD approaches,
    # then at deepest negative value on the HD, goes high positive, then sissles out back to 0
    distance = (-365 / (distance + 1e-10)) # epsilon prevents divbyzero
    if abs(distance) < 365/21:  # clip at +/- 21 days
        distance = 0.
    holi_dist[d] = [nearest, distance]
    if not nearest in all_holidays:
        all_holidays.append(nearest)

all_holidays

In [ ]:
alldays = pd.unique(df['Date'])

# iterate over all dates in use,
# lookup what holiday is nearest and what the distance is (this was precomputed, dunno why anymore)
# stick the distance value in the column named for the holiday
for d in alldays:
    nearest, distance = holi_dist[d]
    # fill the rows with this Date in corresponding holiday-column with the computed "distance"
    df.loc[ df['Date'] == d, nearest] = distance 

# fill NANs with 0s
for holiday in all_holidays:
    df[holiday].fillna(0., inplace=True)  # this appears to miss the tnxg12 column somehow?!?!

# the following may not be good: "A value is trying to be set on a copy of a slice from a DataFrame"
# df[all_holidays].fillna(0, inplace=True) # fill NAs with zeroes

# we might as wel drop IsHoliday from dataframe altogether
# if 'IsHoliday' in df.columns:
#     df.drop('IsHoliday', inplace=True, axis=1)

if "nan" in df.columns:
    print("We have NaNs as column labels now!!")
#     df.drop('nan', inplace=True, axis=1)


In [ ]:
# # show results
# where = (df['Store'] == 1) & (df['Dept'] == 1) 
# # select = ['Weekly_Sales', 'IsHoliday']
# rik = df.loc[where, all_holidays]
# rik.iloc[:10]

In [ ]:
# This clever function adds numerical and boolean fields for every(?) conceivable attribute of a Date object as new colums to df
add_datepart(df, 'Date')
df['Week'][:4]

In [ ]:
# Markdown engineering:

# create 5 new boolean features for the 5 different markdowns
df = df.assign(md1_present = df.MarkDown1.notnull())
df = df.assign(md2_present = df.MarkDown2.notnull())
df = df.assign(md3_present = df.MarkDown3.notnull())
df = df.assign(md4_present = df.MarkDown4.notnull())
df = df.assign(md5_present = df.MarkDown5.notnull())

In [ ]:
# standardize the following numerical columns
stdnums = [
    "Temperature",
    "Unemployment",
    "CPI",
    "Fuel_Price",
    "MarkDown1",
    "MarkDown2",
    "MarkDown3",
    "MarkDown4",
    "MarkDown5",
    "Size",
    'Year', 
    'Month', 
    'Week', 
    'Dayofyear',
#     'xmas09',
    'sbowl10',
    'lday10',
    'tnxg10',
    'xmas10',
    'sbowl11',
    'lday11',
    'tnxg11',
    'xmas11',
    'sbowl12',
    'lday12',
    'tnxg12',
]

for n in stdnums:
    df[n] = (df[n] - df[n].mean())/(df[n].std())

# # rescale the following numerical columns
# normnums = [
#     "Week",
#     "Dayofyear",
#     "Elapsed",
# ]
# for n in normnums:
#     df[n] = (df[n] - df[n].mean()) / abs(1e-16 + df[n].max() - df[n].min())


In [ ]:
# create dummy features/columns for categorical data

catcats = [
#     "Year",
#     "Month",
    "Type",
    "Store",
    "Dept",
]

for c in catcats:
    df[c] = c + df[c].map(str)
    temp_dummies = pd.get_dummies(df[c])
    df = pd.concat([df, temp_dummies], axis=1)
    del df[c]


# one more feature: mean weekly sales

per Store, per Dept, the mean Weekly_Sales is ca 10445. Let's introduce that number as one input in the mix, so the model has something to scale by. Do not standardize/normalize this input!

In [ ]:
df['MWS'] = 10445

In [ ]:
# show results
where = (df['Store1'] == 1) & (df['Dept90'] == 1) 
df.loc[where][:12]

In [ ]:
# plots sales and holidays in time for specific Store/Dept
def plot_store_dept(plotme):
    wsm = plotme.Weekly_Sales.mean()
    fig, ax = plt.subplots(figsize=(13,5))
    ax.set_ylabel("Weekly_Sales")
    ax.bar(x=plotme.index, height=plotme["Weekly_Sales"],width=7, label="known sales")
    ax.axhline(y=wsm, c='c', label="known sales mean")
    ax.plot_date(x=plotme.index, y=(plotme["IsHoliday"] * wsm), fmt='*m')
    ax.legend()
    plt.title(f"Store: {store}\nDept: {dept}")

    plt.show()

In [ ]:
store=1
dept=0

In [ ]:
dept += 1
where = (pristine["Store"] == store) & (pristine["Dept"] == dept) 
select = ['Date', 'Weekly_Sales', 'IsHoliday']
plotme = pristine.loc[where, select]
plotme.set_index('Date', inplace=True)
plot_store_dept(plotme)

In [ ]:
len(df)

In [ ]:
# .isnull() does NOT return zeroes! it returns NaNs
# df.isnull().sum()


In [ ]:
if 'Date' in df.columns:
    df.drop('Date', inplace=True, axis=1)
# df.head()

In [ ]:
df.fillna(0, inplace=True) # fill NAs with zeroes

In [ ]:
train = df.iloc[:len_train]
test = df.iloc[len_train:]

test = test.drop('Weekly_Sales',axis=1) # We should remove the nonsense values from test

y = train['Weekly_Sales'].values
X = train.drop('Weekly_Sales',axis=1).values

In [ ]:
# train.iloc[1]

In [ ]:
# train.to_csv('train_featured.csv',index=False)
# test.to_csv('test_featured.csv',index=False)
# ! ls -l *csv

In [ ]:
# Split separate numpy arrays for train and test
# or, alternatively: order .fit() to take its own splits

# np.random.seed(0)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.0020, random_state=0)
# X=X_train
# y=y_train
# print(X.shape, X_val.shape)
# m_val,n_val = X_val.shape

In [ ]:
# last sanity check
where = (df['Store1'] == 1) & (df['Dept90'] == 1) 
# select = ['Weekly_Sales', 'IsHoliday']
df.loc[where][:12]

In [ ]:
m,n = X.shape

## Tweak your hyper parameters here

In [ ]:
lamb = .01   # regularization rate
drop = 0.10  # dropout rate

In [ ]:
model = keras.models.Sequential()

#, kernel_regularizer = regularizers.l2(lamb)

model.add(Dense(units=n//5, input_dim=n))
# model.add(BatchNormalization())
model.add(Activation('tanh'))
# model.add(Dropout(rate=drop))

model.add(Dense(units=n//10))
# model.add(BatchNormalization())
model.add(Activation('tanh'))
# model.add(Dropout(rate=drop))

model.add(Dense(units=n//10))
# model.add(BatchNormalization())
model.add(Activation('tanh'))
# model.add(Dropout(rate=drop))

model.add(Dense(units=n//15))
# model.add(BatchNormalization())
model.add(Activation('tanh'))
# model.add(Dropout(rate=drop))

model.add(Dense(units=1))
model.add(Activation('linear'))

model.compile(optimizer=optimizers.adam(lr=28), loss='mae')

In [ ]:
model.summary()

## alpha, batchsize, epochs

In [ ]:
# these hyper parameters can be tweaked after compiling the model
# this is useful for retraining an existing model under different params
alpha = 10    # learning rate
bs = min(2**14, m//2)  # batch size: maxed at half size testset
epochs = 5  # number of epochs per training round

In [ ]:
model.optimizer.lr = alpha

In [ ]:
# evaluate model on training and validation sets
print("Loss by Mean Absolute Error")

print("train:      ", model.evaluate(x=X,     y=y,     verbose=1, batch_size=bs))
# print("validation: ", model.evaluate(x=X_val, y=y_val, verbose=0, batch_size=bs))

# start training

In [ ]:
timestart = datetime.datetime.now()

In [ ]:
# def keepon_training():
print("Learning Rate: %f, BatchSize: %i "% (alpha, bs))
history = model.fit(X,y,batch_size=bs,epochs=epochs, validation_split=0.1, verbose =1)
loss = int(model.evaluate(x=X,     y=y,     verbose=0, batch_size=bs))


In [ ]:
# fname = f"nearest_holiday_model_loss_{loss}.h5"
# model.save(fname)

In [ ]:
timedone = datetime.datetime.now()
runtime = timedone - timestart
print(runtime)

# stop training

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss (mae)')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# evaluate model on training and validation sets
# bs_eval = min(m, m_val)
print("Loss by Mean Absolute Error")
print("train:      ", loss)
# print("validation: ", model.evaluate(x=X_val, y=y_val, verbose=0, batch_size=bs_eval))

In [ ]:
X_test = test.values
y_pred = model.predict(X_test,batch_size=bs)
testfile = pd.read_csv('../input/test.csv')
submission = pd.DataFrame({'id':testfile['Store'].map(str) + '_' + testfile['Dept'].map(str) + '_' + testfile['Date'].map(str),
                          'Weekly_Sales':y_pred.flatten()})
submission.to_csv('submission.csv',index=False)

In [ ]:
! ls -l submission.csv
! wc -l submission.csv

In [ ]:
# submission.head(20)

In [ ]:
predicted = testfile.copy()
predicted['Weekly_Prediction'] = y_pred
predicted = pd.concat([predicted,pristine],axis=0) # Join predicted and pristine
# df['time'] = df['time'].astype('datetime64[ns]')
predicted['Date'] = predicted['Date'].astype('datetime64[ns]')
predicted.set_index(['Date'], inplace=True)
# predicted.Weekly_Prediction *= 15 # cause we're desprt
# predicted.index

In [ ]:
# plots sales and predicted sales in time for specific Store/Dept
def plot_store_dept_pred(plotme):
    wpm = int(plotme.Weekly_Prediction.mean())
    wsm = int(plotme.Weekly_Sales.mean())
    fig, ax = plt.subplots(figsize=(13,4))
    ax.set_ylabel("Sales")
    ax.bar(x=plotme.index, height=plotme["Weekly_Sales"],width=7, label="known sales")
    ax.axhline(y=wsm, c='c', label=f"known sales mean {wsm}")
    ax.bar(x=plotme.index, height=plotme["Weekly_Prediction"],width=5, 
           label="predicted sales", color='r')
    ax.axhline(y=wpm, c='g', label=f"predicted sales mean {wpm}")
    ax.plot_date(x=plotme.index, y=(plotme["IsHoliday"] * wsm), fmt='*m')
    ax.legend()
#     ax.xaxis_date()
    plt.title(f"Store: {store}\nDept: {dept}")

    plt.show()

In [ ]:
store=0
dept=10

In [ ]:
store += 1
# dept  += 1

where = (predicted['Store'] == store) & (predicted['Dept'] == dept)
select = ['Date', 'Weekly_Prediction', 'Weekly_Sales', 'IsHoliday']
plotme = predicted.loc[where, select]
plot_store_dept_pred(plotme)
# print(plotme.Weekly_Prediction.mean(),plotme.Weekly_Sales.mean())
# print(plotme.Weekly_Prediction.count(),plotme.Weekly_Sales.count())

In [ ]:
# undertow cells in notebook?

In [ ]:
# undertow cells in notebook?

In [ ]:
# undertow cells in notebook?

In [ ]:
# undertow cells in notebook?

In [ ]:
# undertow cells in notebook?